In [1]:
import pandas as pd
import pympi
import os
import math

In [18]:
language = 'ES' # 'EN' or 'ES'

In [19]:
# Get the list of files in the directory
files = os.listdir('C:\Research\\features\{language}'.format(language=language))

# Initialize an empty list to store the file IDs
file_ids = []

# Iterate through the files
for file in files:
    # Check if the file name ends with '-reduction.eaf'
    if file.endswith('-reduction.eaf'):
        # Extract the file ID from the file name
        file_id = file[:-14]
        # Add the file ID to the list
        file_ids.append(file_id)

# Print the list of file IDs
print(file_ids)


['ES_001', 'ES_003', 'ES_008', 'ES_012', 'ES_022', 'ES_028']


In [20]:
features = ['tl -250 to -100 self',
'tl -100 to  -20 self',
'tl  -20 to   20 self',
'tl    20 to 100 self',
'tl   100 to 250 self',
'th -250 to -100 self',
'th  -100 to -20 self',
'th   -20 to  20 self',
'th    20 to 100 self',
'th   100 to 250 self',
'vo -250 to -100 self',
'vo -100 to  -20 self',
'vo -20 to    20 self',
'vo    20 to 100 self',
'vo   100 to 250 self',
'np -250 to -100 self',
'np -100 to -20 self',
'np  -20 to   20 self',
'np    20 to 100 self',
'np   100 to 250 self',
'wp -250 to -100 self',
'wp -100 to  -20 self',
'wp  -20 to   20 self',
'wp    20 to 100 self',
'wp   100 to 250 self',
'cr -250 to -100 self',
'cr -100 to -20 self',
'cr   -20 to  20 self',
'cr    20 to 100 self',
'cr   100 to 250 self',
'vf -250 to -100 self',
'vf  -100 to -20 self',
'vf   -20 to  20 self',
'vf    20 to 100 self',
'vf   100 to 250 self',
're -250 to -100 self',
're  -100 to -20 self',
're   -20 to  20 self',
're    20 to 100 self',
're   100 to 250 self',
'en -250 to -100 self',
'en  -100 to -20 self',
'en   -20 to  20 self',
'en    20 to 100 self',
'en   100 to 250 self',
'le -250 to -100 self',
'le  -100 to -20 self',
'le   -20 to  20 self',
'le    20 to 100 self',
'le   100 to 250 self',
'sf -250 to -100 self',
'sf  -100 to -20 self',
'sf   -20 to  20 self',
'sf    20 to 100 self',
'sf   100 to 250 self',
'pd -250 to -100 self',
'pd  -100 to -20 self',
'pd   -20 to  20 self',
'pd    20 to 100 self',
'pd   100 to 250 self',
'st -250 to -100 self',
'st  -100 to -20 self',
'st   -20 to  20 self',
'st    20 to 100 self',
'st   100 to 250 self',
'tr -250 to -100 self',
'tr  -100 to -20 self',
'tr   -20 to  20 self',
'tr    20 to 100 self',
'tr   100 to 250 self',
'tf -250 to -100 self',
'tf  -100 to -20 self',
'tf   -20 to  20 self',
'tf    20 to 100 self',
'tf   100 to 250 self',
'tm -250 to -100 self',
'tm  -100 to -20 self',
'tm   -20 to  20 self',
'tm    20 to 100 self',
'tm   100 to 250 self']

In [21]:
total_time_labeled = 0

for file_id in file_ids:

    start_time = math.inf
    end_time = -math.inf

    for side in ['left', 'right']:
        print("Working on {} {} channel".format(file_id, side))
        # Load the CSV file into a dataframe, specifying the header and names arguments
        df = pd.read_csv('C:\\Research\\features\\{language}\\{side}_channel_{file_id}-downsampledfeatures.csv'.format(language=language, side=side, file_id=file_id), header=0, names=features)
        df['label'] = -1

        # Add a column 'sample_id' that contains the row number and the file_id e.g., {row_number}_{file_id} in the same column, in the first column
        df.insert(0, 'sample_id', ['{}_{}_{}'.format(file_id, i,side) for i in range(df.shape[0])])
        # Load the .eaf file using the pympi library
        eaf_file = 'C:\\Research\\features\\{language}\\{file_id}-reduction.eaf'.format(language=language, file_id=file_id)
        eafob = pympi.Elan.Eaf(eaf_file)

        # Get the tier containing the annotations
        tier = eafob.get_annotation_data_for_tier('Reduction {side}'.format(side=side.title()))

        # Get the lower start time and the higher end time
        start_recording_time = min(tier[0][0], start_time)
        end_recording_time = max(tier[-1][1], end_time)

        # Iterate through the annotations on the tier
        for annotation in tier:
            # Get the start and end time of the annotation
            annotation_start_time = annotation[0]
            annotation_end_time = annotation[1]
            total_time_labeled += annotation_end_time - annotation_start_time
            if annotation_start_time//10 < df.shape[0] and annotation_end_time//10 < df.shape[0]:
                df.loc[[i for i in range(annotation_start_time//10, annotation_end_time//10)], 'label'] = annotation[2]

            # Print the start and end time
            # print(f"Start time: {start_time}, End time: {end_time}")

        # Print start and end time
        if side == 'right':
            print("Start time: {}, End time: {}".format(start_recording_time/1000/60, end_recording_time/1000/60))
        
        # Save labeled data as a CSV
        df.to_csv('C:\Research\labeled_features\{language}\\{side}_channel_labeled_features_{file_id}.csv'.format(language=language, side=side, file_id=file_id), index=False)

print("Total time labeled: {}".format(total_time_labeled/1000/60))

Working on ES_001 left channel
Working on ES_001 right channel
Start time: 0.03816666666666667, End time: 5.963833333333333
Working on ES_003 left channel
Working on ES_003 right channel
Start time: 0.006166666666666667, End time: 1.2061666666666668
Working on ES_008 left channel
Working on ES_008 right channel
Start time: 0.126, End time: 1.7156999999999998
Working on ES_012 left channel
Working on ES_012 right channel
Start time: 0.003, End time: 5.021100000000001
Working on ES_022 left channel
Working on ES_022 right channel
Start time: 0.006, End time: 4.998133333333333
Working on ES_028 left channel
Working on ES_028 right channel
Start time: 0.06683333333333333, End time: 4.66435
Total time labeled: 16.884383333333332


# Debugging

In [36]:
# Load the CSV file into a dataframe, specifying the header and names arguments
df = pd.read_csv('C:\\Research\\features\\ES\\left_channel_ES_006-downsampledfeatures.csv', header=0, names=['lp -250 to -100','lp  -100 to -20','lp    -20 to 20','lp    20 to 100','lp   100 to 250','hp -250 to -100','hp  -100 to -20','hp    -20 to 20','hp    20 to 100','hp   100 to 250','np -250 to -100','np  -100 to -20','np    -20 to 20','np    20 to 100','np   100 to 250','wp -250 to -100','wp  -100 to -20','wp    -20 to 20','wp    20 to 100','wp   100 to 250','vo -250 to -100','vo  -100 to -20','vo    -20 to 20','vo    20 to 100','vo   100 to 250'])
df['label'] = -1
# df.head()


,lp -250 to -100,lp -100 to -20,lp -20 to 20,lp 20 to 100,lp 100 to 250,hp -250 to -100,hp -100 to -20,hp -20 to 20,hp 20 to 100,hp 100 to 250,...,wp -100 to -20,wp -20 to 20,wp 20 to 100,wp 100 to 250,vo -250 to -100,vo -100 to -20,vo -20 to 20,vo 20 to 100,vo 100 to 250,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.223288,-0.091211,-0.208290,-1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.273154,-0.084757,-0.192873,-1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.028155,-0.092616,-0.177381,-1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.140455,-0.091854,-0.179985,-1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.191755,-0.079923,-0.169937,-1


In [39]:
# Load the .eaf file using the pympi library
eaf_file = 'C:\\Research\\features\\ES\\ES_006-reduction.eaf'
eafob = pympi.Elan.Eaf(eaf_file)


# Get the tier containing the annotations
tier = eafob.get_annotation_data_for_tier('Reduction Left')

# Iterate through the annotations on the tier
for annotation in tier:
    # Get the start and end time of the annotation
    start_time = annotation[0]
    end_time = annotation[1]
    df.loc[[i for i in range(start_time//10,end_time//10)], 'label'] = annotation[2]

    # Print the start and end time
    print(f"Start time: {start_time}, End time: {end_time}")

df.to_csv('C:\Research\labeled_features\ES\labeled_f_1.csv')

Start time: 3346, End time: 4626
Start time: 6770, End time: 7398
Start time: 7398, End time: 8644
Start time: 8644, End time: 8771
Start time: 8773, End time: 9561
Start time: 12447, End time: 12649
Start time: 16422, End time: 16728
Start time: 16973, End time: 17132
Start time: 17849, End time: 18387
Start time: 21459, End time: 21638
Start time: 22599, End time: 22868
Start time: 22941, End time: 23624
Start time: 25497, End time: 26039
Start time: 30170, End time: 30874
Start time: 32439, End time: 33022
Start time: 33024, End time: 33417
Start time: 33668, End time: 33953
Start time: 34015, End time: 34925
Start time: 34969, End time: 35557
Start time: 35557, End time: 35936
Start time: 36137, End time: 36558
Start time: 36955, End time: 37154
Start time: 37154, End time: 37310
Start time: 37721, End time: 38086
Start time: 38087, End time: 38270
Start time: 38709, End time: 38930
Start time: 38955, End time: 39412
Start time: 39414, End time: 39920
Start time: 40652, End time: 4

In [10]:
import pandas as pd

# Create a list of numbers from 1 to 20
numbers = list(range(1, 21))

# Divide the list into four columns
col1 = numbers[0:5]
col2 = numbers[5:10]
col3 = numbers[10:15]
col4 = numbers[15:20]

# Create the dataframe with the four columns
df = pd.DataFrame({'col1': col1, 'col2': col2, 'col3': col3, 'col4': col4})

# Print the dataframe
print(df)

   col1  col2  col3  col4
0     1     6    11    16
1     2     7    12    17
2     3     8    13    18
3     4     9    14    19
4     5    10    15    20


In [32]:
df.loc[[i for i in range(1,4)], 'col2'] = 99

,col1,col2,col3,col4
0,1,6,11,16
1,2,99,12,17
2,3,99,13,18
3,4,99,14,19
4,5,10,15,20
